In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-5.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0     prop-5      347    1    1    0    1    5     0   0   1  ...  0.000000   
1     prop-5      347    1    1    0    0    5     0   0   0  ...  0.000000   
2     prop-5      347    3    1    0   13   33     3   0  13  ...  0.142857   
3     prop-5      347    1    1    0    0    5     0   0   0  ...  0.000000   
4     prop-5      347    1    1    0    1    5     0   0   1  ...  0.000000   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
8511  prop-5      362    3    3    0    8   14     1   0   8  ...  0.500000   
8512  prop-5      362    2    3    0   17   17     1   0  17  ...  0.000000   
8513  prop-5      362    2    3    0   12   15     1   0  12  ...  0.000000   
8514  prop-5      362    2    3    0   15   20     1   0  15  ...  0.000000   
8515  prop-5      362   40    1    0    7   48   780   0   7  ...  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
1       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
2       0  0.000000  0.666667   0    0  46.333333       4  1.6667    0  
3       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
4       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
...   ...       ...       ...  ..  ...        ...     ...     ...  ...  
8511    0  0.947368  0.666667   0    0  13.666667       2  0.6667    0  
8512    0  0.857143  0.625000   1    2  25.000000       1  0.5000    0  
8513    0  0.857143  0.625000   1    1  21.000000       1  0.5000    1  
8514    0  0.857143  0.625000   1    1  41.000000       3  1.5000    0  
8515    0  0.000000  0.616667   0    0   8.425000       3  1.0500    8  

[8516 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 347]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0     prop-5      347    1    1    0    1    5     0   0   1  ...  0.000000   
1     prop-5      347    1    1    0    0    5     0   0   0  ...  0.000000   
2     prop-5      347    3    1    0   13   33     3   0  13  ...  0.142857   
3     prop-5      347    1    1    0    0    5     0   0   0  ...  0.000000   
4     prop-5      347    1    1    0    1    5     0   0   1  ...  0.000000   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
2866  prop-5      347    4    3    0   11   46     0   0  11  ...  0.200000   
2867  prop-5      347    2    3    0    9   12     1   0   9  ...  0.000000   
2868  prop-5      347    3    3    0    8   14     1   0   8  ...  0.500000   
2869  prop-5      347    2    3    0   29   22     1   0  29  ...  0.000000   
2870  prop-5      347    2    4    0   15   10     1   0  15  ...  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
1       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
2       0  0.000000  0.666667   0    0  46.333333       4  1.6667    0  
3       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
4       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
...   ...       ...       ...  ..  ...        ...     ...     ...  ...  
2866    0  0.875000  0.833333   2    3  83.000000       6  1.7500    0  
2867    0  0.857143  0.625000   1    1  29.000000       3  1.5000    1  
2868    0  0.947368  0.666667   0    0  13.666667       2  0.6667    0  
2869    0  0.857143  0.625000   1    1  66.000000       6  3.0000    0  
2870    0  0.928571  0.625000   1    1  15.000000       1  0.5000    0  

[2871 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
2339    6    4    0    7   39    13   0   7    5  0.960000  181  0.200000   
1295    4    5    0    2    9     6   0   2    2  1.111111   45  0.000000   
1153    2    1    0   17   20     1   1  16    2  2.000000   69  0.000000   
2768    2    4    0   16   12     1   0  16    2  2.000000   42  0.000000   
825     8    1    0    6    8    28   3   3    8  2.000000    8  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
2706    7    5    0   37   51    13   0  37    3  0.937500  332  0.000000   
1249    2    3    0   19   24     1   0  19    2  1.000000  135  0.000000   
1536    2    3    0   12   11     1   0  12    2  2.000000   40  0.000000   
702     5    4    0    7   24     8   1   6    2  0.666667  176  0.666667   
21      1    1    0    0    2     0   0   0    1  2.000000   47  0.000000   

      moa       mfa    cam  ic  cbm        amc  max_cc  avg_cc  
2339    0  0.826087  0.600   1    1  28.333333       5  1.6667  
1295    0  0.920000  1.000   2    2   9.500000       1  0.5000  
1153    0  0.000000  0.625   0    0  33.500000       1  0.5000  
2768    0  0.928571  0.625   1    1  20.000000       1  0.5000  
825     0  0.000000  0.300   0    0   0.000000       1  1.0000  
...   ...       ...    ...  ..  ...        ...     ...     ...  
2706    0  0.800000  0.400   3    5  45.285714       4  1.1429  
1249    0  0.857143  0.625   1    1  64.500000       1  0.5000  
1536    0  0.857143  0.625   1    1  19.000000       1  0.5000  
702     0  0.911111  0.450   3    4  33.600000       5  1.6000  
21      0  0.000000  1.000   0    0   3.000000       0  0.0000  

[2296 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc  dam  moa  \
305     8    5    0   23   42    16   2  21    7  0.904762   207  0.0    0   
92      4    4    0   19   50     4   1  18    2  0.833333   273  0.5    0   
195     0    1    0    0    0     0   0   0    0  2.000000     0  0.0    0   
2421    6    5    0   24   28     9   0  24    2  0.933333   101  0.0    0   
887     9    4    0   31   59     0   0  31    2  0.625000  1094  0.0    0   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...  ...  ...   
2535    2    3    0   18   16     1   0  18    2  1.000000    56  0.0    0   
1462    2    2    0   11   13     1   0  11    2  1.000000    64  0.0    0   
1174    6    4    0   13   17     9   0  13    2  0.950000    82  0.0    0   
73     11    1    0    3   13     0   2   1   11  0.100000    74  1.0    1   
1954    2    1    0    8    6     1   1   7    2  2.000000    16  0.0    0   

           mfa       cam  ic  cbm         amc  max_cc  avg_cc  
305   0.818182  0.607143   3    4   24.500000       1  0.7500  
92    0.931818  0.437500   3    6   66.750000       7  2.5000  
195   0.000000  0.000000   0    0    0.000000       0  0.0000  
2421  0.833333  0.600000   3    4   15.333333       1  0.6667  
887   0.619048  0.611111   1    1  120.222222       1  0.8889  
...        ...       ...  ..  ...         ...     ...     ...  
2535  0.857143  0.625000   1    1   26.500000       2  1.0000  
1462  0.800000  0.666667   1    1   30.000000       3  1.5000  
1174  0.826087  0.600000   2    4   12.000000       1  0.6667  
73    0.000000  0.500000   0    0    5.636364       1  0.9091  
1954  0.000000  0.625000   0    0    7.000000       1  0.5000  

[575 rows x 20 columns]

In [9]:
y_train = train['bug']
y_test = test['bug']

In [10]:
y_train

2339    1
1295    0
1153    1
2768    0
825     0
       ..
2706    1
1249    0
1536    0
702     0
21      0
Name: bug, Length: 2296, dtype: int64

In [11]:
y_test

305     0
92      0
195     0
2421    0
887     0
       ..
2535    0
1462    0
1174    0
73      0
1954    0
Name: bug, Length: 575, dtype: int64

In [12]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [13]:
x_train_np

array([[ 6.        ,  4.        ,  0.        , ..., 28.33333333,
         5.        ,  1.6667    ],
       [ 4.        ,  5.        ,  0.        , ...,  9.5       ,
         1.        ,  0.5       ],
       [ 2.        ,  1.        ,  0.        , ..., 33.5       ,
         1.        ,  0.5       ],
       ...,
       [ 2.        ,  3.        ,  0.        , ..., 19.        ,
         1.        ,  0.5       ],
       [ 5.        ,  4.        ,  0.        , ..., 33.6       ,
         5.        ,  1.6       ],
       [ 1.        ,  1.        ,  0.        , ...,  3.        ,
         0.        ,  0.        ]])

In [14]:
x_test_np

array([[ 8.        ,  5.        ,  0.        , ..., 24.5       ,
         1.        ,  0.75      ],
       [ 4.        ,  4.        ,  0.        , ..., 66.75      ,
         7.        ,  2.5       ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 6.        ,  4.        ,  0.        , ..., 12.        ,
         1.        ,  0.6667    ],
       [11.        ,  1.        ,  0.        , ...,  5.63636364,
         1.        ,  0.9091    ],
       [ 2.        ,  1.        ,  0.        , ...,  7.        ,
         1.        ,  0.5       ]])

In [15]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [16]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [17]:
train_x

array([[[ 6.        ,  4.        ,  0.        , ..., 28.33333333,
          5.        ,  1.6667    ]],

       [[ 4.        ,  5.        ,  0.        , ...,  9.5       ,
          1.        ,  0.5       ]],

       [[ 2.        ,  1.        ,  0.        , ..., 33.5       ,
          1.        ,  0.5       ]],

       ...,

       [[ 2.        ,  3.        ,  0.        , ..., 19.        ,
          1.        ,  0.5       ]],

       [[ 5.        ,  4.        ,  0.        , ..., 33.6       ,
          5.        ,  1.6       ]],

       [[ 1.        ,  1.        ,  0.        , ...,  3.        ,
          0.        ,  0.        ]]])

In [18]:
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [20]:
y = list(test_y)

In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv347/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
23/23 [==============================] - 9s 33ms/step - loss: 0.3179 - mse: 0.3179 - mae: 0.1361 - root_mean_squared_error: 0.5638 - mean_squared_logarithmic_error: 0.0492
Epoch 2/100
23/23 [==============================] - 0s 21ms/step - loss: 0.3139 - mse: 0.3139 - mae: 0.1678 - root_mean_squared_error: 0.5603 - mean_squared_logarithmic_error: 0.0493
Epoch 3/100
23/23 [==============================] - 0s 21ms/step - loss: 0.3117 - mse: 0.3117 - mae: 0.1496 - root_mean_squared_error: 0.5583 - mean_squared_logarithmic_error: 0.0478
Epoch 4/100
23/23 [==============================] - 0s 21ms/step - loss: 0.3069 - mse: 0.3069 - mae: 0.1543 - root_mean_squared_error: 0.5540 - mean_squared_logarithmic_error: 0.0472
Epoch 5/100
23/23 [==============================] - 0s 22ms/step - loss: 0.3000 - mse: 0.3000 - mae: 0.1504 - root_mean_squared_error: 0.5478 - mean_squared_logarithmic_error: 0.0458
Epoch 6/100
23/23 [==============================] - 0s 21ms/step - lo

23/23 [==============================] - 1s 23ms/step - loss: 0.1504 - mse: 0.1504 - mae: 0.0955 - root_mean_squared_error: 0.3878 - mean_squared_logarithmic_error: 0.0306
Epoch 89/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1461 - mse: 0.1461 - mae: 0.0995 - root_mean_squared_error: 0.3823 - mean_squared_logarithmic_error: 0.0317
Epoch 90/100
23/23 [==============================] - 1s 25ms/step - loss: 0.1501 - mse: 0.1501 - mae: 0.0932 - root_mean_squared_error: 0.3874 - mean_squared_logarithmic_error: 0.0319
Epoch 91/100
23/23 [==============================] - 1s 24ms/step - loss: 0.1457 - mse: 0.1457 - mae: 0.1030 - root_mean_squared_error: 0.3817 - mean_squared_logarithmic_error: 0.0303
Epoch 92/100
23/23 [==============================] - 1s 23ms/step - loss: 0.1478 - mse: 0.1478 - mae: 0.0955 - root_mean_squared_error: 0.3844 - mean_squared_logarithmic_error: 0.0303
Epoch 93/100
23/23 [==============================] - 1s 22ms/step - loss: 0.1454 - mse:

23/23 [==============================] - 0s 21ms/step - loss: 0.1523 - mse: 0.1523 - mae: 0.1135 - root_mean_squared_error: 0.3903 - mean_squared_logarithmic_error: 0.0332
Epoch 74/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1469 - mse: 0.1469 - mae: 0.1069 - root_mean_squared_error: 0.3833 - mean_squared_logarithmic_error: 0.0317
Epoch 75/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1491 - mse: 0.1491 - mae: 0.1034 - root_mean_squared_error: 0.3862 - mean_squared_logarithmic_error: 0.0322
Epoch 76/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1502 - mse: 0.1502 - mae: 0.1064 - root_mean_squared_error: 0.3875 - mean_squared_logarithmic_error: 0.0324
Epoch 77/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1500 - mse: 0.1500 - mae: 0.1006 - root_mean_squared_error: 0.3872 - mean_squared_logarithmic_error: 0.0324
Epoch 78/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2201 - mse:

23/23 [==============================] - 0s 22ms/step - loss: 0.1586 - mse: 0.1586 - mae: 0.1150 - root_mean_squared_error: 0.3983 - mean_squared_logarithmic_error: 0.0321
Epoch 59/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1543 - mse: 0.1543 - mae: 0.1076 - root_mean_squared_error: 0.3929 - mean_squared_logarithmic_error: 0.0307
Epoch 60/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1536 - mse: 0.1536 - mae: 0.1029 - root_mean_squared_error: 0.3919 - mean_squared_logarithmic_error: 0.0311
Epoch 61/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1517 - mse: 0.1517 - mae: 0.1068 - root_mean_squared_error: 0.3895 - mean_squared_logarithmic_error: 0.0304
Epoch 62/100
23/23 [==============================] - 1s 22ms/step - loss: 0.1493 - mse: 0.1493 - mae: 0.1038 - root_mean_squared_error: 0.3863 - mean_squared_logarithmic_error: 0.0307
Epoch 63/100
23/23 [==============================] - 0s 22ms/step - loss: 0.1510 - mse:

Epoch 1/100
23/23 [==============================] - 7s 37ms/step - loss: 0.3191 - mse: 0.3191 - mae: 0.1319 - root_mean_squared_error: 0.5649 - mean_squared_logarithmic_error: 0.0495
Epoch 2/100
23/23 [==============================] - 0s 21ms/step - loss: 0.3139 - mse: 0.3139 - mae: 0.1633 - root_mean_squared_error: 0.5603 - mean_squared_logarithmic_error: 0.0489
Epoch 3/100
23/23 [==============================] - 0s 21ms/step - loss: 0.3109 - mse: 0.3109 - mae: 0.1564 - root_mean_squared_error: 0.5576 - mean_squared_logarithmic_error: 0.0481
Epoch 4/100
23/23 [==============================] - 0s 21ms/step - loss: 0.3085 - mse: 0.3085 - mae: 0.1541 - root_mean_squared_error: 0.5554 - mean_squared_logarithmic_error: 0.0480
Epoch 5/100
23/23 [==============================] - 0s 21ms/step - loss: 0.3023 - mse: 0.3023 - mae: 0.1501 - root_mean_squared_error: 0.5498 - mean_squared_logarithmic_error: 0.0462
Epoch 6/100
23/23 [==============================] - 0s 21ms/step - loss: 0.2936

...

23/23 [==============================] - 0s 22ms/step - loss: 0.1498 - mse: 0.1498 - mae: 0.1050 - root_mean_squared_error: 0.3870 - mean_squared_logarithmic_error: 0.0321
Epoch 90/100
23/23 [==============================] - 0s 22ms/step - loss: 0.1467 - mse: 0.1467 - mae: 0.1168 - root_mean_squared_error: 0.3831 - mean_squared_logarithmic_error: 0.0314
Epoch 91/100
23/23 [==============================] - 1s 23ms/step - loss: 0.1413 - mse: 0.1413 - mae: 0.1071 - root_mean_squared_error: 0.3759 - mean_squared_logarithmic_error: 0.0309
Epoch 92/100
23/23 [==============================] - 1s 22ms/step - loss: 0.1393 - mse: 0.1393 - mae: 0.0934 - root_mean_squared_error: 0.3732 - mean_squared_logarithmic_error: 0.0294
Epoch 93/100
23/23 [==============================] - 1s 23ms/step - loss: 0.1367 - mse: 0.1367 - mae: 0.0973 - root_mean_squared_error: 0.3697 - mean_squared_logarithmic_error: 0.0300
Epoch 94/100
23/23 [==============================] - 1s 23ms/step - loss: 0.1401 - mse:

23/23 [==============================] - 0s 21ms/step - loss: 0.1499 - mse: 0.1499 - mae: 0.1046 - root_mean_squared_error: 0.3872 - mean_squared_logarithmic_error: 0.0321
Epoch 75/100
23/23 [==============================] - 0s 22ms/step - loss: 0.1445 - mse: 0.1445 - mae: 0.1075 - root_mean_squared_error: 0.3801 - mean_squared_logarithmic_error: 0.0314
Epoch 76/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1467 - mse: 0.1467 - mae: 0.0989 - root_mean_squared_error: 0.3831 - mean_squared_logarithmic_error: 0.0307
Epoch 77/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1515 - mse: 0.1515 - mae: 0.1122 - root_mean_squared_error: 0.3892 - mean_squared_logarithmic_error: 0.0325
Epoch 78/100
23/23 [==============================] - 0s 21ms/step - loss: 0.1478 - mse: 0.1478 - mae: 0.1045 - root_mean_squared_error: 0.3844 - mean_squared_logarithmic_error: 0.0315
Epoch 79/100
23/23 [==============================] - 0s 20ms/step - loss: 0.1455 - mse:

23/23 [==============================] - 0s 20ms/step - loss: 0.3240 - mse: 0.3240 - mae: 0.0923 - root_mean_squared_error: 0.5692 - mean_squared_logarithmic_error: 0.0511
Epoch 60/100
23/23 [==============================] - 0s 19ms/step - loss: 0.3240 - mse: 0.3240 - mae: 0.0923 - root_mean_squared_error: 0.5692 - mean_squared_logarithmic_error: 0.0511
Epoch 61/100
23/23 [==============================] - 0s 19ms/step - loss: 0.3240 - mse: 0.3240 - mae: 0.0923 - root_mean_squared_error: 0.5692 - mean_squared_logarithmic_error: 0.0511
Epoch 62/100
23/23 [==============================] - 0s 21ms/step - loss: 0.3240 - mse: 0.3240 - mae: 0.0923 - root_mean_squared_error: 0.5692 - mean_squared_logarithmic_error: 0.0511
Epoch 63/100
23/23 [==============================] - 1s 22ms/step - loss: 0.3240 - mse: 0.3240 - mae: 0.0923 - root_mean_squared_error: 0.5692 - mean_squared_logarithmic_error: 0.0511
Epoch 64/100
23/23 [==============================] - 1s 22ms/step - loss: 0.3240 - mse:

Epoch 1/100
23/23 [==============================] - 10s 74ms/step - loss: 0.3187 - mse: 0.3187 - mae: 0.1284 - root_mean_squared_error: 0.5646 - mean_squared_logarithmic_error: 0.0493
Epoch 2/100
23/23 [==============================] - 1s 24ms/step - loss: 0.3141 - mse: 0.3141 - mae: 0.1699 - root_mean_squared_error: 0.5604 - mean_squared_logarithmic_error: 0.0494
Epoch 3/100
23/23 [==============================] - 1s 24ms/step - loss: 0.3118 - mse: 0.3118 - mae: 0.1544 - root_mean_squared_error: 0.5584 - mean_squared_logarithmic_error: 0.0482
Epoch 4/100
23/23 [==============================] - 1s 23ms/step - loss: 0.3076 - mse: 0.3076 - mae: 0.1545 - root_mean_squared_error: 0.5546 - mean_squared_logarithmic_error: 0.0477
Epoch 5/100
23/23 [==============================] - 1s 24ms/step - loss: 0.3027 - mse: 0.3027 - mae: 0.1525 - root_mean_squared_error: 0.5502 - mean_squared_logarithmic_error: 0.0467
Epoch 6/100
23/23 [==============================] - 1s 23ms/step - loss: 0.293

23/23 [==============================] - 1s 25ms/step - loss: 0.1558 - mse: 0.1558 - mae: 0.1132 - root_mean_squared_error: 0.3947 - mean_squared_logarithmic_error: 0.0332
Epoch 90/100
23/23 [==============================] - 1s 26ms/step - loss: 0.1513 - mse: 0.1513 - mae: 0.1103 - root_mean_squared_error: 0.3889 - mean_squared_logarithmic_error: 0.0317
Epoch 91/100
23/23 [==============================] - 1s 25ms/step - loss: 0.1554 - mse: 0.1554 - mae: 0.1065 - root_mean_squared_error: 0.3941 - mean_squared_logarithmic_error: 0.0305
Epoch 92/100
23/23 [==============================] - 1s 25ms/step - loss: 0.1559 - mse: 0.1559 - mae: 0.1159 - root_mean_squared_error: 0.3949 - mean_squared_logarithmic_error: 0.0333
Epoch 93/100
23/23 [==============================] - 1s 25ms/step - loss: 0.1530 - mse: 0.1530 - mae: 0.1120 - root_mean_squared_error: 0.3911 - mean_squared_logarithmic_error: 0.0321
Epoch 94/100
23/23 [==============================] - 1s 26ms/step - loss: 0.1493 - mse:

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv347/'

In [23]:
total_training_time

[65.30218720436096,
 62.170448303222656,
 62.25294637680054,
 63.57427740097046,
 65.59286069869995,
 66.03131365776062,
 69.89120078086853,
 64.21412420272827,
 67.9620954990387,
 74.73629117012024]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 22.05759150981903


In [25]:
total_training_loss_history

[{'loss': [0.31789878010749817,
   0.313916951417923,
   0.31165796518325806,
   0.306928813457489,
   0.3000452518463135,
   0.2915542721748352,
   0.28093060851097107,
   0.28056371212005615,
   0.26188987493515015,
   0.2497481256723404,
   0.24041299521923065,
   0.2617245316505432,
   0.23329229652881622,
   0.22613586485385895,
   0.23237620294094086,
   0.22841328382492065,
   0.20624446868896484,
   0.21124765276908875,
   0.20691029727458954,
   0.19541960954666138,
   0.19794133305549622,
   0.20377518236637115,
   0.19371680915355682,
   0.19158229231834412,
   0.20309028029441833,
   0.19053992629051208,
   0.18758666515350342,
   0.22978226840496063,
   0.18290391564369202,
   0.19778227806091309,
   0.18388685584068298,
   0.1909920871257782,
   0.19997026026248932,
   0.1890629380941391,
   0.1924297958612442,
   0.18103143572807312,
   0.17812518775463104,
   0.18191835284233093,
   0.1705651730298996,
   0.168296679854393,
   0.1782342493534088,
   0.16928020119667053,

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.0574654499689738
avg mae of training: 0.035936338206132255


In [27]:
total_scores

[[0.30473288893699646,
  0.30473288893699646,
  0.1371006816625595,
  0.5520261526107788,
  0.052666421979665756],
 [0.3613915741443634,
  0.3613915741443634,
  0.12029169499874115,
  0.6011584997177124,
  0.05717853084206581],
 [0.37476053833961487,
  0.37476053833961487,
  0.18916456401348114,
  0.6121768951416016,
  0.06565526127815247],
 [0.2534897029399872,
  0.2534897029399872,
  0.15122826397418976,
  0.5034776329994202,
  0.04927343502640724],
 [0.3450157046318054,
  0.3450157046318054,
  0.12390833348035812,
  0.5873803496360779,
  0.058309778571128845],
 [0.331120103597641,
  0.331120103597641,
  0.1442549079656601,
  0.5754303932189941,
  0.057801611721515656],
 [0.30686327815055847,
  0.30686327815055847,
  0.1526428610086441,
  0.5539523959159851,
  0.05587558075785637],
 [0.2757847309112549,
  0.2757847309112549,
  0.16452431678771973,
  0.5251520872116089,
  0.05606003478169441],
 [0.2800000011920929,
  0.2800000011920929,
  0.09913043677806854,
  0.5291502475738525,
  0

In [1]:
a = [[0.30473288893699646,
  0.30473288893699646,
  0.1371006816625595,
  0.5520261526107788,
  0.052666421979665756],
 [0.3613915741443634,
  0.3613915741443634,
  0.12029169499874115,
  0.6011584997177124,
  0.05717853084206581],
 [0.37476053833961487,
  0.37476053833961487,
  0.18916456401348114,
  0.6121768951416016,
  0.06565526127815247],
 [0.2534897029399872,
  0.2534897029399872,
  0.15122826397418976,
  0.5034776329994202,
  0.04927343502640724],
 [0.3450157046318054,
  0.3450157046318054,
  0.12390833348035812,
  0.5873803496360779,
  0.058309778571128845],
 [0.331120103597641,
  0.331120103597641,
  0.1442549079656601,
  0.5754303932189941,
  0.057801611721515656],
 [0.30686327815055847,
  0.30686327815055847,
  0.1526428610086441,
  0.5539523959159851,
  0.05587558075785637],
 [0.2757847309112549,
  0.2757847309112549,
  0.16452431678771973,
  0.5251520872116089,
  0.05606003478169441],
 [0.2800000011920929,
  0.2800000011920929,
  0.09913043677806854,
  0.5291502475738525,
  0.05488220974802971],
 [0.3027208149433136,
  0.3027208149433136,
  0.11492358893156052,
  0.5502007007598877,
  0.055446185171604156]]

In [2]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [3]:
test_mae

[0.1371006816625595,
 0.12029169499874115,
 0.18916456401348114,
 0.15122826397418976,
 0.12390833348035812,
 0.1442549079656601,
 0.1526428610086441,
 0.16452431678771973,
 0.09913043677806854,
 0.11492358893156052]

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.1045293112595876
avg mae: 0.04657232165336609
avg are: 0.022202898313601813


In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.1498594433069229
median testing mse: 0.30579808354377747


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.1109921745955944
median testing mae: 0.1406777948141098


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.0669565200805664


In [35]:
test_ARE

[0.067826085,
 0.07130435,
 0.064347826,
 0.0626087,
 0.069565214,
 0.069565214,
 0.067826085,
 0.064347826,
 0.0626087,
 0.066086955]